In [285]:
#!pip install colorlog PyHive mlrun kubernetes

In [286]:
from os import path, getenv
from mlrun import new_project, mlconf

#project_name = '-'.join(filter(None, ['getting-started-iris', getenv('V3IO_USERNAME', None)]))
project_name = "parquez"
project_path = path.abspath('./')
project = new_project(project_name, project_path)
project.save()
print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/parquez
Project name: parquez


In [287]:
out = mlconf.artifact_path or path.abspath('./data')
# {{run.uid}} will be substituted with the run id, so output will be written to different directoried per run
artifact_path = path.join(out, '{{run.uid}}')

In [288]:
%env PYTHONPATH=./

env: PYTHONPATH=./


In [289]:
project.set_function("functions/validate_input.py", 'validate', kind='job', image='aviaigz/parquez')

In [290]:
project.set_function("functions/get_table_schema.py", 'get_schema', kind='job', image='aviaigz/parquez')

In [291]:
project.set_function("functions/create_parquet_table.py", 'create_parquet', kind='job', image='aviaigz/parquez')

In [292]:
project.set_function("functions/create_kv_view.py", 'create_kv_view', kind='job', image='aviaigz/parquez')

In [293]:
project.set_function("functions/create_unified_view.py", 'create_unified_view', kind='job', image='aviaigz/parquez')

In [294]:
project.set_function("functions/run_parquez_interval.py", 'run_parquez_interval', kind='job', image='aviaigz/parquez')
project.func('run_parquez_interval').apply(mount_v3io())
project.func('run_parquez_interval').set_env('PYTHONPATH', project_path)
project.func('run_parquez_interval').spec.artifact_path = 'User/artifacts'
project.func('run_parquez_interval').spec.service_account='mlrun-api'
project.func('run_parquez_interval').save()

> 2020-07-27 11:44:54,564 [warning] warning!, server (0.4.10) and client (0.5.0) ver dont match
> 2020-07-27 11:44:54,566 [debug] saving function: run-parquez-interval, tag: 


'a42666cc782bbd959c7563704311fe078786ac7f'

In [295]:
project.set_function("functions/run_scheduler.py", 'run_scheduler', kind='job', image='aviaigz/parquez')

<a id="gs-step-create-n-run-ml-pipeline"></a>
## Create and Run a Fully Automated ML Pipeline

You're now ready to create a full ML pipeline.
This is done by using [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/), which is integrated into the Iguazio Data Science Platform.
Kubeflow Pipelines is an open-source framework for building and deploying portable, scalable machine-learning workflows based on Docker containers.
MLRun leverages this framework to take your existing code and deploy it as steps in the pipeline.

In [296]:
%%writefile {path.join(project_path, 'workflow.py')}

from kfp import dsl
from mlrun import mount_v3io
from os import path

funcs = {}


parquez_params = {'view_name':'view_name'
         ,'partition_by':'h'
         ,'partition_interval':'1h'
         ,'real_time_window':'1d'
         ,'historical_retention':'7d'
         ,'real_time_table_name':'faker'
         ,'config_path':'/User/parquez/config/parquez.ini'}


# Configure function resources and local settings
def init_functions(functions: dict, project=None, secrets=None):
    project_path = path.abspath('./')
    for f in functions.values():
        f.apply(mount_v3io())
        f.set_env('PYTHONPATH', project_path)
        f.spec.artifact_path = 'User/artifacts'
        f.spec.service_account='mlrun-api'
        
        
# Create a Kubeflow Pipelines pipeline
@dsl.pipeline(
    name = "parquez-pipeline",
    description = "parquez description"
)
def kfpipeline():
    # Ingest the data set
    validate = funcs['validate'].as_step(
        name="validate",
        params=parquez_params,
        outputs=['validate']
    )
    
    # Analyze the dataset
    schema = funcs['get_schema'].as_step(
        name="get_schema",
        params = parquez_params,
        inputs={'table': validate.outputs},                       
        outputs=['schema']
    )
    
    parquet = funcs["create_parquet"].as_step(
        name="create_parquet",
        params=parquez_params,
        inputs={"table": schema.outputs['schema']},
        outputs=['create_parquet']
    )
    
    kv_view = funcs["create_kv_view"].as_step(
        name="create_kv_view",
        params=parquez_params,
        inputs={'table': parquet.outputs},
        outputs=['kv_view']
    )
    
    unified_view = funcs["create_unified_view"].as_step(
        name="create_unified_view",
        params=parquez_params,
        inputs={'table': kv_view.outputs},
        outputs=['unified_view']
    )
    
    unified_view = funcs["run_scheduler"].as_step(
        name="run_scheduler",
        params=parquez_params,
        inputs={'table': unified_view.outputs},
        outputs=['run_scheduler']
    )    

Overwriting /User/parquez/workflow.py


<a id="gs-register-workflow"></a>
#### Register the Workflow

Use the `set_workflow` MLRun project method to register your workflow with MLRun.
The following code sets the `name` parameter to the selected workflow name ("main") and the `code` parameter to the name of the workflow file that is found in your project directory (**workflow.py**).

In [297]:
# Register the workflow file as "main"
project.set_workflow('main', 'workflow.py')

In [298]:
project.save()

In [299]:
run_id = project.run(
    'main',
    arguments={}, 
    
    artifact_path=path.abspath(path.join('pipeline','{{workflow.uid}}'),
    
                              )
    ,dirty=True)

> 2020-07-27 11:44:54,759 [warning] warning!, server (0.4.10) and client (0.5.0) ver dont match
> 2020-07-27 11:44:54,779 [debug] saving function: validate, tag: 
> 2020-07-27 11:44:54,823 [warning] warning!, server (0.4.10) and client (0.5.0) ver dont match
> 2020-07-27 11:44:54,840 [debug] saving function: get-schema, tag: 
> 2020-07-27 11:44:54,877 [warning] warning!, server (0.4.10) and client (0.5.0) ver dont match
> 2020-07-27 11:44:54,892 [debug] saving function: create-parquet, tag: 
> 2020-07-27 11:44:54,929 [warning] warning!, server (0.4.10) and client (0.5.0) ver dont match
> 2020-07-27 11:44:54,945 [debug] saving function: create-kv-view, tag: 
> 2020-07-27 11:44:54,984 [warning] warning!, server (0.4.10) and client (0.5.0) ver dont match
> 2020-07-27 11:44:54,999 [debug] saving function: create-unified-view, tag: 
> 2020-07-27 11:44:55,037 [warning] warning!, server (0.4.10) and client (0.5.0) ver dont match
> 2020-07-27 11:44:55,052 [debug] saving function: run-scheduler

> 2020-07-27 11:44:55,232 [info] Pipeline run id=891ba404-94cc-4373-a3a6-bc2031412053, check UI or DB for progress
